## 3. Data Is Gathered

Data: 12 de Junho de 2023

Esse notebook contém:


### Notas:



In [54]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [55]:
from nbconf import *

In [56]:
import sqlite3

In [57]:
conn = sqlite3.connect(os.path.join(config.DATA_PATH, config.WEBPAGES_DATABASE))

In [58]:
cursor = conn.cursor()

In [59]:
df = pd.DataFrame(cursor.execute("SELECT * FROM metadata"))

### Data Selection

In [60]:
df.columns = [
    'url',
    'doi',
    'type',
    'author',
    'institute',
    'knowledge_area',
    'committee',
    'title_pt',
    'title_en',
    'keywords_pt',
    'keywords_en',
    'abstract_pt',
    'abstract_en',
    'publish_date'
]

In [61]:
df.head(2)

,url,doi,type,author,institute,knowledge_area,committee,title_pt,title_en,keywords_pt,keywords_en,abstract_pt,abstract_en,publish_date
0,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-06072016-171200,Tese de Doutorado,"Pinto, Irajá Damiani (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Corais carboníferos da Amazônia,Not available,Coral (gema orgânica) Estratigrafia - Amazônia...,Not available,O presente trabalho compreende o estudo de cor...,Not available,2016-07-07
1,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-06072016-172400,Tese de Doutorado,"Kawashita, Koji (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Metodo Rb-Sr em rochas sedimentares: aplicação...,Not available,Geocronologia Rochas sedimentares,Not available,Estudo radiométrico em sedimentos com posição ...,Not available,2016-07-07


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104710 entries, 0 to 104709
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   url             104710 non-null  object
 1   doi             104515 non-null  object
 2   type            104705 non-null  object
 3   author          104705 non-null  object
 4   institute       104705 non-null  object
 5   knowledge_area  104705 non-null  object
 6   committee       100960 non-null  object
 7   title_pt        104634 non-null  object
 8   title_en        102793 non-null  object
 9   keywords_pt     104498 non-null  object
 10  keywords_en     100533 non-null  object
 11  abstract_pt     104633 non-null  object
 12  abstract_en     103690 non-null  object
 13  publish_date    104620 non-null  object
dtypes: object(14)
memory usage: 11.2+ MB


#### Selection

In [63]:
data = df.loc[:,["title_en", "abstract_en", "keywords_en"]]
metadata = df.loc[:, ["url", "type", "author", "institute", "title_en", "abstract_en", "keywords_en"]]

#### Null values

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104710 entries, 0 to 104709
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title_en     102793 non-null  object
 1   abstract_en  103690 non-null  object
 2   keywords_en  100533 non-null  object
dtypes: object(3)
memory usage: 2.4+ MB


In [65]:
data = data.replace(to_replace="Not available", value=None)
# data = data.replace(to_replace="não possui", value=None)
# data = data.replace(to_replace="Não disponível", value=None)
# data = data.replace(to_replace="No abstract", value=None)

In [66]:
data = data[data['title_en'].notnull() & data['abstract_en'].notnull()]
idx = data.index

In [67]:
metadata = metadata.iloc[idx]

In [68]:
data = data.reset_index(drop=True)
metadata = metadata.reset_index(drop=True)

In [69]:
data.iloc[10000]

title_en       Effect of the use of Probiotic on parameters o...
abstract_en    The demand from consumers for good-quality pro...
keywords_en    Broilers Hematology Immunity Performance Probi...
Name: 10000, dtype: object

In [70]:
metadata.iloc[10000]

url            https://www.teses.usp.br/teses/disponiveis/10/...
type                                     Dissertação de Mestrado
author               Bittencourt, Letícia Cardoso (Catálogo USP)
institute          Faculdade de Medicina Veterinária e Zootecnia
title_en       Effect of the use of Probiotic on parameters o...
abstract_en    The demand from consumers for good-quality pro...
keywords_en    Broilers Hematology Immunity Performance Probi...
Name: 10000, dtype: object

In [71]:
data = data.fillna("")

In [23]:
data.to_csv(os.path.join(config.DATA_PATH, "clean_data.csv" ), escapechar='\\')

### Encoding

O código a seguir cria batches de descrições a partir do título, abstract e palavras-chave e solicita ao modelo que codifique as embeddings, que são apensadas a vectors ao final.

##### Exemplo de execução

In [72]:
row = data.iloc[97601,:]
pre = (row.title_en + ". " + row.abstract_en + " " + row.keywords_en).split(". ")
pre

['Mitigating DDoS attacks on IoT through machine learning and network functions virtualization',
 'The Internet of Things (IoT) has undergone a rapid popularization, reaching a wide range of application domains',
 'As a consequence, more and more IoT devices with diverse characteristics are deployed in a variety of public and private environments, progressively becoming common objects of everyday life',
 'On the other hand, the physical infrastructure of heterogeneous systems is complex and requires efficient and dynamic solutions for managing network performance and security, at a level that allows standardized deployment and easy replication in smart industries, buildings and cities',
 'An approach that has been gaining ground when the intention is to respond to security threats on IoT is the Network Functions Virtualization (NFV) usage',
 'The literature presents some works that use NFV to detect and mitigate security threats on IoT networks, but few performance indicators of the ne

In [49]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device='cuda')

NameError: name 'SentenceTransformer' is not defined

In [217]:
model.encode(pre).shape

(12, 768)

In [51]:
# batch of size 64
batch = [pre for _ in range(64)]
import time
s = time.time()
vectors = model.encode(batch)
e = time.time()
print(e - s, " segundos")

NameError: name 'model' is not defined

In [50]:
vectors.shape

NameError: name 'vectors' is not defined

In [187]:
# o texto é representado por
print(len(vectors[0]), "embeddings")

768 embeddings


#### Cálculo dos vetores

In [222]:
# encoding in batches can reduce overhead costs and speed up the process
vectors = []
batch_size = 64  # TODO: how can I speed up? Maybe reduce batch?
batch = []
for row in tqdm(data.itertuples()):
    descriptions = []
    pre = (row.title_en + ". " + row.abstract_en + " " + row.keywords_en).split(". ")
    encoding = model.encode(pre)
    vectors.append(encoding)
#     if len(batch) >= batch_size:
#         vectors.append(model.encode(batch))
#         batch = []
# if len(batch) > 0:
#     vectors.append(model.encode(batch))
#     batch = []

vectors_concat = np.concatenate(vectors)

0it [00:00, ?it/s]

In [73]:
idxs = []
for i, row in enumerate(data.itertuples()):
    for _ in range(len((row.title_en + ". " + row.abstract_en + " " + row.keywords_en).split(". "))):
        idxs.append(i)

In [75]:
import pickle

In [76]:
import os

In [79]:
with open(os.path.join(config.DATA_PATH, "idxs.pickle"), 'wb') as f:
    pickle.dump(idxs, f)

In [225]:
vectors_concat

array([[-0.4340217 ,  0.42884657,  1.4143783 , ..., -0.15943804,
        -0.85209644, -0.7908082 ],
       [ 0.23602659,  0.20836525,  0.99276525, ...,  0.19000989,
        -0.4924341 , -0.8562151 ],
       [ 0.16545534, -0.09567884,  1.3298293 , ...,  0.08211403,
        -0.50803363, -0.40309826],
       ...,
       [-0.32332236,  0.5328971 ,  0.69116515, ...,  0.41696966,
        -0.20721593,  0.18184057],
       [-0.8625958 ,  0.6534884 ,  0.20543995, ..., -0.30923805,
        -0.56630987, -0.19633113],
       [ 0.88802785,  0.18949236,  0.15269034, ...,  0.18557502,
         0.249683  ,  1.4947641 ]], dtype=float32)

In [226]:
np.save(os.path.join(config.DATA_PATH,"vectors_experiment_4.npy"), vectors_concat, allow_pickle=False)

### Testes - Para ter certeza de que os vetores fazem sentido

In [227]:
vectors = np.load(os.path.join(config.DATA_PATH,"vectors_experiment_4.npy"))

In [228]:
from sklearn.metrics.pairwise import cosine_similarity

> **Pergunta**: Como funciona cosine_similarity?

In [238]:
import time
time.time()

1686621456.7033858

In [239]:
import time
def query(sample_query):
    s = time.time()
    query_vector = model.encode(sample_query)
    e0 = time.time()
    print(f"Encode took {e0-s}s")
    
    e1 = time.time()
    [scores] = cosine_similarity([query_vector], vectors)
    e2 = time.time()
    print(f"Encode took {e2-e1}s")
    
    top_scores_ids = np.argsort(scores)
    top_scores_ids = top_scores_ids[::-1][:15]

    for top_id in top_scores_ids:
        tmp = metadata.iloc[idxs[top_id]]
        print("Index:", idxs[top_id])
        print("Title: ", tmp.title_en)
        print("Author: ", tmp.author)
        print("Abstract: ", tmp.abstract_en)
        print("Keywords: ", tmp.keywords_en)
        print("\n##############################################################################\n")

In [203]:
query("ddos attack")

Index: 97601
Title:  Mitigating DDoS attacks on IoT through machine learning and network functions virtualization
Author:  Oliveira, Guilherme Werneck de (Catálogo USP)
Abstract:  The Internet of Things (IoT) has undergone a rapid popularization, reaching a wide range of application domains. As a consequence, more and more IoT devices with diverse characteristics are deployed in a variety of public and private environments, progressively becoming common objects of everyday life. On the other hand, the physical infrastructure of heterogeneous systems is complex and requires efficient and dynamic solutions for managing network performance and security, at a level that allows standardized deployment and easy replication in smart industries, buildings and cities. An approach that has been gaining ground when the intention is to respond to security threats on IoT is the Network Functions Virtualization (NFV) usage. The literature presents some works that use NFV to detect and mitigate secur

In [205]:
query("ddos attack using machine learning")

Index: 27488
Title:  Machine learning in complex networks
Author:  Breve, Fabricio Aparecido (Catálogo USP)
Abstract:  Complex networks is a recent and active scientific research field, which concerns large scale networks with non-trivial topological structure, such as computer networks, telecommunication networks, transport networks, social networks and biological networks. Many of these networks are naturally divided into communities or modules and, therefore, uncovering their structure is one of the main problems related to complex networks study. This problem is related with the machine learning field, which is concerned with the design and development of algorithms and techniques which allow computers to learn, or increase their performance based on experience. Some of the problems identified in traditional learning techniques include: difficulties in identifying irregular forms in the attributes space; uncovering overlap structures of groups or classes, which occurs when elements

In [240]:
query("ddos attack")

Encode took 0.14992904663085938s
Encode took 275.84169483184814s
Index: 53793
Title:  Method for mitigating against distributed denial of service attacks using multi-agent system.
Author:  Pereira, João Paulo Aragão (Catálogo USP)
Abstract:  The quality of service offered by the Internet Service Provider (ISP) depends directly on the amount of resources available at that time. In recent decades, this quality has been affected by the frequent and intense attacks that consume these resources, such as the Distributed Denial of Service (DDoS) attacks. In order to make the ISPs network more resilient to different types of DDoS attacks, techniques have been developed against such attacks over the past few years. Aiming to contribute to the improvement of such mechanisms, this dissertation presents a reactive autonomous method for detecting and mitigating DDoS attacks using a Multi-Agent system (MAS), in networks of ISPs. The main property of the proposed method is to identify characteristic 

In [241]:
query("ddos attack using machine learning")

Encode took 0.15382623672485352s
Encode took 356.5039610862732s
Index: 97601
Title:  Mitigating DDoS attacks on IoT through machine learning and network functions virtualization
Author:  Oliveira, Guilherme Werneck de (Catálogo USP)
Abstract:  The Internet of Things (IoT) has undergone a rapid popularization, reaching a wide range of application domains. As a consequence, more and more IoT devices with diverse characteristics are deployed in a variety of public and private environments, progressively becoming common objects of everyday life. On the other hand, the physical infrastructure of heterogeneous systems is complex and requires efficient and dynamic solutions for managing network performance and security, at a level that allows standardized deployment and easy replication in smart industries, buildings and cities. An approach that has been gaining ground when the intention is to respond to security threats on IoT is the Network Functions Virtualization (NFV) usage. The literatu